In [1]:
import sys
sys.path.append('../../30_data_tools/')

In [2]:
import pandas as pd
import sqlite3
from datetime import datetime
import uuid

In [3]:
from helper import load_dotenv
from get_labelstudio_data import get_results_of_project

In [4]:
from tqdm import tqdm

In [5]:
dotenv = load_dotenv()

In [6]:
con = sqlite3.connect(dotenv['DB_PATH'])

In [7]:
def load_data( pkl_path ):
    data = pd.read_pickle(pkl_path)
    data.loc[
        :,
        'timestamp'
    ] = datetime.fromtimestamp( int(pkl_path.name.strip(pkl_path.suffix)) )

    return data

In [8]:
def get_temp_id():
    myuuid = uuid.uuid4()
    myuuidStr = 'temp_' + str(myuuid)
    
    return myuuidStr

In [12]:
data = pd.concat([
    load_data(pkl_path) for pkl_path in dotenv['GENERIC_INFORMATION_DATA_DIR'].glob('./*.pkl')
]).reset_index().drop(columns=['index'])

data.loc[:,'job'] = data.img_path.apply(lambda val: val.parent.parent.name)
data.loc[:,'pdf_filename'] = data.img_path.apply(lambda val: val.name.replace('.4c.jpg',''))
data.loc[:,'idx'] = data.basic_name.str.extract(r'.+\$PLACEHOLDER\$\.(.+)')
data.loc[:,'variant_name'] = f'halftone{ dotenv["LOFI_DPI"] }dpi'
data.loc[:,'type'] = '4c'
data.loc[:,'img_name'] = data.apply(lambda row: row.basic_name.replace("$PLACEHOLDER$",row.method) + '.' + row["type"] + '.jpg', axis=1)
data.loc[:,'bbox_string'] = data.bbox.apply(lambda val: ";".join([str(v) for v in val]))

data.overlay_intensity_C = data.overlay_intensity_C.fillna(0)
data.overlay_intensity_M = data.overlay_intensity_M.fillna(0)
data.overlay_intensity_Y = data.overlay_intensity_Y.fillna(0)
data.overlay_intensity_K = data.overlay_intensity_K.fillna(0.7)

data.use_blow_up_region = data.use_blow_up_region.fillna(False)
data.use_contract_region = data.use_contract_region.fillna(False)
data.use_wave_deform = data.use_wave_deform.fillna(False)

In [105]:
data

,image,img_path,method,ssim,bbox,use_blow_up_centered,use_contract_centered,use_pillow_disortion,use_roll,use_rotation,...,contract_center,contract_c,job,pdf_filename,idx,variant_name,type,img_name,bbox_string,id
0,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.809063,"[1536, 4810, 1392, 990]",False,False,False,False,False,...,NaN,NaN,606023,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,1,halftone600dpi,4c,606023.tg39_0086_20-1257_Part07_03_PGUE_GS_Pre...,1536;4810;1392;990,NaN
1,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.880812,"[4116, 5362, 864, 384]",False,False,False,False,False,...,NaN,NaN,606023,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,1,halftone600dpi,4c,606023.tg39_0086_20-1257_Part07_03_PGUE_GS_Pre...,4116;5362;864;384,NaN
2,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.883277,"[354, 5026, 1254, 744]",False,False,False,False,False,...,NaN,NaN,606023,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,1,halftone600dpi,4c,606023.tg39_0086_20-1257_Part07_03_PGUE_GS_Pre...,354;5026;1254;744,NaN
3,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.999449,"[2442, 4960, 1038, 786]",False,False,False,False,False,...,NaN,NaN,606023,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,1,halftone600dpi,4c,606023.tg39_0086_20-1257_Part07_03_PGUE_GS_Pre...,2442;4960;1038;786,NaN
4,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.816406,"[4542, 3702, 606, 1422]",False,False,False,False,False,...,NaN,NaN,606023,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,1,halftone600dpi,4c,606023.tg39_0086_20-1257_Part07_03_PGUE_GS_Pre...,4542;3702;606;1422,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,CJ134_046047.p2.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.822905,"[2286, 3048, 792, 1590]",False,False,False,False,True,...,NaN,NaN,615311,CJ134_046047.p2,1,halftone600dpi,4c,615311.CJ134_046047.p2.soft_light.1.4c.jpg,2286;3048;792;1590,NaN
6561,CJ134_046047.p2.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.841398,"[2916, 3216, 426, 402]",False,False,False,False,True,...,NaN,NaN,615311,CJ134_046047.p2,1,halftone600dpi,4c,615311.CJ134_046047.p2.soft_light.1.4c.jpg,2916;3216;426;402,NaN
6562,CJ134_046047.p2.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.907596,"[2154, 354, 744, 636]",False,False,False,False,True,...,NaN,NaN,615311,CJ134_046047.p2,1,halftone600dpi,4c,615311.CJ134_046047.p2.soft_light.1.4c.jpg,2154;354;744;636,NaN
6563,CJ134_046047.p2.4c,/Users/frederic.birwe/Library/CloudStorage/One...,soft_light,0.828713,"[150, 2172, 648, 240]",False,False,False,False,True,...,NaN,NaN,615311,CJ134_046047.p2,1,halftone600dpi,4c,615311.CJ134_046047.p2.soft_light.1.4c.jpg,150;2172;648;240,NaN


In [15]:
results = get_results_of_project(2)
results = pd.DataFrame.from_dict(results)
results.loc[:,'bbox_string'] = results.value.apply(lambda val: f'{ val["x"] };{ val["y"] };{ val["width"] };{ val["height"] }')

In [16]:
results.loc[
    pd.isna(results.id),
    'id'
] = results.loc[pd.isna(results.id)].apply(lambda row: get_temp_id(), axis=1)

In [99]:
results.iloc[4000].img_name

'615311.CJ134_034035.p1.soft_light.1.4c_600.jpg'

In [95]:
data.img_name.iloc[0]

'606023.tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_0321_DE.p1.soft_light.1.4c.jpg'

In [104]:
data.loc[
    data.img_name.str.contains('615311.CJ134_034035.p1.soft_light.1.4c')
].img_name.iloc[0]

'615311.CJ134_034035.p1.soft_light.1.4c.jpg'

In [86]:
data.loc[
    pd.isna(data.id)
].shape

(6565, 53)

In [17]:
data = pd.merge(
    data,
    results.loc[
        :,
        ['img_name','bbox_string','id']
    ],
    how="left",
    on=['img_name','bbox_string']
)

In [56]:
def filter_already_processed( data, sql_string, con ):
    # bestehende generic images auslesen
    already_processed_df = pd.read_sql( sql_string, con )

    already_processed = []
    for _,row in already_processed_df.iterrows():
        already_processed.append(";".join([str(v) for v in row.values]))

    # relevant rows auf neue limitieren
    return data.loc[
        data.apply(
            lambda row: ";".join([str(v) for v in row.values]),
            axis=1
        ).isin(already_processed) == False
    ]

# Generic images schreiben

In [61]:
def write_generic_images( data, con ):
    relevant_rows = data.loc[
        :,
        ['pdf_filename','job','type','variant_name','method','idx','timestamp']
    ].groupby(['pdf_filename','job','type','variant_name','method','idx']).last().reset_index()

    relevant_rows = filter_already_processed(
        relevant_rows,
        '''
            SELECT pdf_filename, job,"type",variant_name,method,idx,timestamp
            FROM generic_image 
        ''',
        con
        
    )
    
    # Bilder schreiben
    sql_lines = [
        f"('{ row.pdf_filename }','{ row.job }','{ row.type }','{ row.variant_name }','{ row.method }',{ row.idx },'{ row.timestamp }')"
        for _,row in relevant_rows.iterrows()
    ]

    if len(sql_lines) > 0:
        SQL = f'''
            INSERT INTO generic_image ('pdf_filename','job','type','variant_name','method','idx','timestamp')
            VALUES { ",".join(sql_lines) }
        '''
        
        c = con.cursor()
        c.execute(SQL)
        c.close()
        con.commit()

In [62]:
write_generic_images( data, con )

# Masks schreiben

In [74]:
def write_masks( data, con ):
    relevant_rows = data.loc[
        :,
        [
            'pdf_filename',
            'job',
            'type',
            'variant_name',
            'method',
            'idx',
            'id',
            'bbox_string',
            'ssim',
            'overlay_intensity_C',
            'overlay_intensity_M',
            'overlay_intensity_Y',
            'overlay_intensity_K'
        ]
    ].groupby(['pdf_filename','job','type','variant_name','method','idx','id']).last().reset_index()

    return relevant_rows
    
    relevant_rows = filter_already_processed(
        relevant_rows,
        '''
            SELECT
                'pdf_filename',
                'job',
                'type',
                'variant_name',
                'method',
                'idx',
                'mask_id',
                'bbox',
                'ssim',
                'overlay_intensity_C',
                'overlay_intensity_M',
                'overlay_intensity_Y',
                'overlay_intensity_K'
            FROM mask
        ''',
        con
    )
    
    sql_lines = [
        f"('{ row.pdf_filename }','{ row.job }','{ row.type }','{ row.variant_name }','{ row.method }',{ row.idx },'{ row['id'] }','{ row.bbox_string }',{ row.ssim },{ row.overlay_intensity_C },{ row.overlay_intensity_M },{ row.overlay_intensity_Y },{ row.overlay_intensity_K })"
        for _,row in relevant_rows.iterrows()
    ]

    if len(sql_lines) > 0:
        SQL = f'''
            INSERT INTO mask ('pdf_filename','job','type','variant_name','method','idx','mask_id','bbox','ssim','overlay_intensity_C','overlay_intensity_M','overlay_intensity_Y','overlay_intensity_K')
            VALUES { ",".join(sql_lines) }
        '''
        
        c = con.cursor()
        c.execute(SQL)
        c.close()
        con.commit()

In [77]:
data.loc[
    :,
    [
        'pdf_filename',
        'job',
        'type',
        'variant_name',
        'method',
        'idx',
        'id',
        'bbox_string',
        'ssim',
        'overlay_intensity_C',
        'overlay_intensity_M',
        'overlay_intensity_Y',
        'overlay_intensity_K'
    ]
].groupby(['pdf_filename','job','type','variant_name','method','idx','id']).last().reset_index()

,pdf_filename,job,type,variant_name,method,idx,id,bbox_string,ssim,overlay_intensity_C,overlay_intensity_M,overlay_intensity_Y,overlay_intensity_K
0,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,606023,4c,halftone600dpi,soft_light,1,NaN,1536;4810;1392;990,0.809063,0.0,0.0,0.0,0.7
1,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,606023,4c,halftone600dpi,soft_light,1,NaN,4116;5362;864;384,0.880812,0.0,0.0,0.0,0.7
2,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,606023,4c,halftone600dpi,soft_light,1,NaN,354;5026;1254;744,0.883277,0.0,0.0,0.0,0.7
3,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,606023,4c,halftone600dpi,soft_light,1,NaN,2442;4960;1038;786,0.999449,0.0,0.0,0.0,0.7
4,tg39_0086_20-1257_Part07_03_PGUE_GS_Premium_03...,606023,4c,halftone600dpi,soft_light,1,NaN,4542;3702;606;1422,0.816406,0.0,0.0,0.0,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6560,CJ134_046047.p2,615311,4c,halftone600dpi,soft_light,1,NaN,2286;3048;792;1590,0.822905,0.0,0.0,0.0,1.0
6561,CJ134_046047.p2,615311,4c,halftone600dpi,soft_light,1,NaN,2916;3216;426;402,0.841398,0.0,0.0,0.0,1.0
6562,CJ134_046047.p2,615311,4c,halftone600dpi,soft_light,1,NaN,2154;354;744;636,0.907596,0.0,0.0,0.0,1.0
6563,CJ134_046047.p2,615311,4c,halftone600dpi,soft_light,1,NaN,150;2172;648;240,0.828713,0.0,0.0,0.0,1.0


In [75]:
write_masks( data, con )

,pdf_filename,job,type,variant_name,method,idx,id,bbox_string,ssim,overlay_intensity_C,overlay_intensity_M,overlay_intensity_Y,overlay_intensity_K


# Adjustments schreiben

In [67]:
def get_features_by_row( row ):
    features = []
    
    if row.use_blow_up_centered:
        features.append((
            'blow_up_centered',
            {
                "centered_c" : row.rotation_degree
            }
        ))

    if row.use_blow_up_region:
        features.append((
            'blow_up_region',
            {
                "blow_up_count" : row.blow_up_count,
                "blow_up_radius" : row.blow_up_radius,
                "blow_up_center" : row.blow_up_center,
                "blow_up_c" : row.blow_up_c,
            }
        ))

    if row.use_contract_region:
        features.append((
            'contract_region',
            {
                "contract_count" : row.contract_count,
                "contract_radius" : row.contract_radius,
                "contract_center" : row.contract_center,
                "contract_c" : row.contract_c,
            }
        ))

    if row.use_contract_centered:
        features.append((
            'contract_centered',
            {
                "centered_c" : row.rotation_degree
            }
        ))

    if row.use_pillow_disortion:
        features.append((
            'pillow_disortion',
            {
                "pillow_depth_x" : row.pillow_depth_x,
                "pillow_depth_y" : row.pillow_depth_y
            }
        ))

    if row.use_roll:
        features.append((
            'roll',
            {
                "pattern_stretch_factor" : row.pattern_stretch_factor
            }
        ))
    
    if row.use_rotation:
        features.append((
            'rotation',
            {
                "rotation_degree" : row.rotation_degree
            }
        ))

    if row.use_scale:
        features.append((
            'scale',
            {
                "scale" : row.scale
            }
        ))


    if row.use_stretch:
        features.append((
            'stretch',
            {
                "stretch_x" : row.stretch_x,
                "stretch_y" : row.stretch_y
            }
        ))


    if row.use_trapezoidal_distortion:
        features.append((
            'trapezoidal_distortion',
            {
                "trapezoidal_distortion_strength_1" : row.trapezoidal_distortion_strength_1,
                "trapezoidal_distortion_strength_2" : row.trapezoidal_distortion_strength_2,
                "trapezoidal_distortion_direction" : row.trapezoidal_distortion_direction
            }
        ))

    if row.use_uniform_trapezoidal_distortion:
        features.append((
            'uniform_trapezoidal_distortion',
            {
                "trapezoidal_distortion_strength" : row.trapezoidal_distortion_strength,
                "trapezoidal_distortion_direction" : row.trapezoidal_distortion_direction
            }
        ))


    if row.use_wave_deform:
        features.append((
            'wave_deform',
            {
                "wave_length" : row.wave_length,
                "wave_depth" : row.wave_depth
            }
        ))

    return features

In [69]:
sql_lines = []

for i in tqdm(range(relevant_rows.shape[0])):
    r = relevant_rows.iloc[i]

    row = data.loc[
        (data.pdf_filename == r.pdf_filename) &
        (data.job == r.job) &
        (data.type == r.type) &
        (data.variant_name == r.variant_name) &
        (data.method == r.method) &
        (data.idx == r.idx) &
        (data.id == r.id)
    ].iloc[0]

    features = get_features_by_row( row )

    for j in range(len(features)):
        f = features[j]

        sql_lines.append(
            f"('{ row.pdf_filename }','{ row.job }','{ row.type }','{ row.variant_name }','{ row.method }',{ row.idx },'{ row['id'] }','{ f[0] }','{ json.dumps(f[1]) }')"

        )

0it [00:00, ?it/s]


In [70]:
sql_lines

[]

In [ ]:
import json

In [ ]:
import re

In [ ]:
SQL = f'''
    INSERT INTO adjustment_per_mask ('pdf_filename','job','type','variant_name','method','idx','mask_id','adjustment','features')
    VALUES { ",".join(sql_lines[:3]) }
'''

c = con.cursor()
c.execute(SQL)
c.close()
con.commit()